In [1]:
import os

curr_folder = os.getcwd() + "/"
datasets_folder = curr_folder[:-6] + "Datasets/"
datasets_folder

'/home/ipsilva/Repositories/dynamic-projections/Datasets/'

In [2]:
datasets_subfolders = []
for i in os.walk(datasets_folder):
    datasets_subfolders.append(i[0])

datasets_subfolders = datasets_subfolders[1:]
datasets_subfolders

['/home/ipsilva/Repositories/dynamic-projections/Datasets/gaussians',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/quickdraw',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/sorts',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/walk',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/nnset',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/fashion',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/cifar10cnn',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/esc50',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/qtables',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/cartolastd']

In [3]:
dataset_number = 1
selected_dataset = datasets_subfolders[dataset_number]
selected_dataset

'/home/ipsilva/Repositories/dynamic-projections/Datasets/quickdraw'

In [4]:
from os import listdir
from os.path import isfile, join

files = [f for f in listdir(selected_dataset) if isfile(join(selected_dataset, f))]
files[:5]

['cello-115665-5.png',
 'bicycle-8656-2.png',
 'cello-555-48.png',
 'baseball-117977-20.png',
 'cello-73920-38.png']

In [5]:
from sklearn.preprocessing import normalize
from itertools import product
import pandas as pd
import numpy as np
import cv2
import re

images_as_lines = []

obs_per_timestep = {}
for file in files:
    file_loc = selected_dataset + "/" + file
    image = cv2.imread(file_loc)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    line = normalize(gray, axis=0, norm='max').reshape(-1)
    images_as_lines.append(line)
    
    time = re.search("[0-9]+.png$", file).group()[:-4]
    try:
        obs_per_timestep[time] == False
        obs_per_timestep[time] += 1
    except:
        obs_per_timestep[time] = 1

obs_per_timestep = list(obs_per_timestep.values())
median_obs_per_timestep = np.median(obs_per_timestep)
dev_obs_per_timestep = np.std(obs_per_timestep)

(i,j) = gray.shape
i_l = [i_e for i_e in range(i)]
j_l = [j_e for j_e in range(j)]
columns = list(product(i_l, j_l))        
df = pd.DataFrame(images_as_lines, columns=columns)
print(df.shape, df.head())

(21670, 784)    (0, 0)  (0, 1)  (0, 2)  (0, 3)  (0, 4)  (0, 5)  (0, 6)  (0, 7)  (0, 8)  \
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     0.0     0.0     0.0     0.0     1.0     1.0     1.0     1.0     1.0   

   (0, 9)  ...  (27, 18)  (27, 19)  (27, 20)  (27, 21)  (27, 22)  (27, 23)  \
0     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
1     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
2     1.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
3     1.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
4     1.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   

   (27, 24)  (27, 25)  (27, 26)  (27, 27)  
0       0.0

In [6]:
from sklearn.decomposition import PCA

def intrisic_dim(dataframe, target_variance):
    pca = PCA()

    principalComponents = pca.fit_transform(df)

    list_variance = pca.explained_variance_ratio_

    final_var = 0
    dims = 0
    for var in list_variance:
        final_var += var
        dims += 1
        if final_var > target_variance:
            break
    return dims, final_var, principalComponents[:dims]

In [7]:
target_variance = 0.9

(dims, final_var, components) = intrisic_dim(df, target_variance)

print("Total time-steps:", len(obs_per_timestep))
print("Registers per time-step (median):", median_obs_per_timestep)
print("Registers per time-step (deviation):", dev_obs_per_timestep)
print("Total dimensions:", df.shape[1])
print("Intrinsic dimensions:", dims)
print("Total variance:", final_var)

Total time-steps: 89
Registers per time-step (median): 130.0
Registers per time-step (deviation): 243.3966159117527
Total dimensions: 784
Intrinsic dimensions: 332
Total variance: 0.900248141125839
